In [ ]:
import pandas as pd
import numpy as np
import os
import logging
from scipy.stats import norm

In [ ]:
trades_df = pd.read_csv("/content/sample_data/TRADES_CopyTr_90D_ROI.csv")
print(trades_df.head())
print(trades_df.info())

              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None


In [ ]:
print(type(trades_df['Trade_History'].loc[0]))

<class 'str'>


In [ ]:

import ast  # Importing ast for safe evaluation of strings

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Data Loading and Exploration

def clean_trade_data(trade_data):
    cleaned_trades = []

    if isinstance(trade_data, str):  # Parse the string using ast.literal_eval
        try:
            trade_list = ast.literal_eval(trade_data) # Safely evaluate the string
            if isinstance(trade_list, list): # Check if the evaluated object is a list
                for i, trade in enumerate(trade_list):
                    try:
                        original_time = trade.get('time')
                        new_trade = trade.copy()

                        try:  # Time conversion
                            new_trade['time'] = pd.to_datetime(trade['time'], unit='ms')
                            if pd.isna(new_trade['time']):
                                raise ValueError("Invalid or out-of-range timestamp")
                        except (ValueError, pd.errors.OutOfBoundsDatetime) as e:
                            logging.error(f"Trade {i} (original time: {original_time}): Time conversion error: {e}. Skipping.")
                            continue

                        try:  # realizedProfit conversion
                            new_trade['realizedProfit'] = float(trade['realizedProfit'])
                        except (ValueError, TypeError) as e:
                            logging.error(f"Trade {i} (original time: {original_time}): realizedProfit conversion error: {e}. Skipping.")
                            continue

                        cleaned_trades.append(new_trade)

                    except Exception as e:
                        logging.error(f"Unexpected error processing trade {i}: {e}. Skipping trade.")
                        continue
            elif isinstance(trade_list, dict): # Check if the evaluated object is a dictionary
                try:
                    original_time = trade_list.get('time')
                    new_trade = trade_list.copy()

                    try:  # Time conversion
                        new_trade['time'] = pd.to_datetime(trade_list['time'], unit='ms')
                        if pd.isna(new_trade['time']):
                            raise ValueError("Invalid or out-of-range timestamp")
                    except (ValueError, pd.errors.OutOfBoundsDatetime) as e:
                        logging.error(f"Single trade (original time: {original_time}): Time conversion error: {e}. Skipping.")
                        return []

                    try:  # realizedProfit conversion
                        new_trade['realizedProfit'] = float(trade_list['realizedProfit'])
                    except (ValueError, TypeError) as e:
                        logging.error(f"Single trade (original time: {original_time}): realizedProfit conversion error: {e}. Skipping.")
                        return []

                    cleaned_trades.append(new_trade)

                except Exception as e:
                    logging.error(f"Unexpected error processing single trade: {e}. Skipping trade.")
                    return []
            else:
                logging.warning(f"Unexpected evaluated structure: {type(trade_list)}. Skipping.")
                pass
        except (SyntaxError, ValueError) as e:  # Catch evaluation errors
            logging.error(f"Error evaluating string: {e}. Skipping.")
            return [] # Return empty if can't evaluate

    elif isinstance(trade_data, list): # Check if it's already a list (no evaluation needed)
        for i, trade in enumerate(trade_data):
            try:
                original_time = trade.get('time')
                new_trade = trade.copy()

                try:  # Time conversion
                    new_trade['time'] = pd.to_datetime(trade['time'], unit='ms')
                    if pd.isna(new_trade['time']):
                        raise ValueError("Invalid or out-of-range timestamp")
                except (ValueError, pd.errors.OutOfBoundsDatetime) as e:
                    logging.error(f"Trade {i} (original time: {original_time}): Time conversion error: {e}. Skipping.")
                    continue

                try:  # realizedProfit conversion
                    new_trade['realizedProfit'] = float(trade['realizedProfit'])
                except (ValueError, TypeError) as e:
                    logging.error(f"Trade {i} (original time: {original_time}): realizedProfit conversion error: {e}. Skipping.")
                    continue

                cleaned_trades.append(new_trade)

            except Exception as e:
                logging.error(f"Unexpected error processing trade {i}: {e}. Skipping trade.")
                continue

    elif isinstance(trade_data, dict):
        try:
            original_time = trade_data.get('time')
            new_trade = trade_data.copy()

            try:  # Time conversion
                new_trade['time'] = pd.to_datetime(trade_data['time'], unit='ms')
                if pd.isna(new_trade['time']):
                    raise ValueError("Invalid or out-of-range timestamp")
            except (ValueError, pd.errors.OutOfBoundsDatetime) as e:
                logging.error(f"Single trade (original time: {original_time}): Time conversion error: {e}. Skipping.")
                return []

            try:  # realizedProfit conversion
                new_trade['realizedProfit'] = float(trade_data['realizedProfit'])
            except (ValueError, TypeError) as e:
                logging.error(f"Single trade (original time: {original_time}): realizedProfit conversion error: {e}. Skipping.")
                return []

            cleaned_trades.append(new_trade)

        except Exception as e:
            logging.error(f"Unexpected error processing single trade: {e}. Skipping trade.")
            return []

    elif isinstance(trade_data, (int, float, np.number)):
        logging.warning(f"Trade_History entry is a number: {trade_data}. Skipping.")
        pass
    else:
        logging.warning(f"Unexpected data type in Trade_History: {type(trade_data)}. Skipping.")
        pass

    return cleaned_trades


trades_df['Trade_History'] = trades_df['Trade_History'].apply(clean_trade_data)

# ... (Rest of the code - same as before) ...

In [ ]:
# 3. Feature Engineering and Metric Calculation
def calculate_metrics(group):
    all_trades = []
    for trade_list in group['Trade_History']:
        all_trades.extend(trade_list) # flatten the list of lists
    group_metrics = pd.Series(index=['ROI','PnL','Sharpe Ratio','MDD','Win Rate','Win Positions','Total Positions'])
    if not all_trades: # if list is empty, default to 0
        group_metrics[:] = 0
        return group_metrics
    trades_df_temp = pd.DataFrame(all_trades) # Create a temp dataframe
    total_positions = len(trades_df_temp)
    win_positions = len(trades_df_temp[trades_df_temp['realizedProfit'] > 0])
    win_rate = (win_positions / total_positions) if total_positions > 0 else 0

    # Calculate ROI (Return on Investment)
    initial_balance = 1  # Assuming a starting balance of 1 for simplicity (you may use a different value if you have it)
    final_balance = initial_balance + trades_df_temp['realizedProfit'].sum()
    roi = ((final_balance - initial_balance) / initial_balance) * 100

    # Calculate PnL (Profit and Loss)
    pnl = trades_df_temp['realizedProfit'].sum()

    # Calculate Sharpe Ratio (assuming daily returns)
    daily_returns = trades_df_temp.set_index('time')['realizedProfit'].resample('D').sum() # Resample to daily
    average_daily_return = daily_returns.mean()
    std_dev_daily_return = daily_returns.std()
    sharpe_ratio = np.sqrt(252) * average_daily_return / std_dev_daily_return if std_dev_daily_return > 0 else 0 # Assuming 252 trading days in a year

    # Calculate MDD (Maximum Drawdown)
    cumulative_returns = (1 + daily_returns.fillna(0)).cumprod() # Fillna to handle potential NaNs from resampling
    peak = cumulative_returns.max()
    drawdown = (cumulative_returns - peak) / peak
    mdd = drawdown.min()

    return pd.Series({
        'ROI': roi,
        'PnL': pnl,
        'Sharpe Ratio': sharpe_ratio,
        'MDD': mdd,
        'Win Rate': win_rate,
        'Win Positions': win_positions,
        'Total Positions': total_positions
    })

metrics_df = trades_df.groupby('Port_IDs').apply(calculate_metrics).reset_index()

<ipython-input-76-090d0c4ecdf2>:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  metrics_df = trades_df.groupby('Port_IDs').apply(calculate_metrics).reset_index()


In [ ]:
# 4. Ranking and Top 20 List (same as before)
weights = {
    'ROI': 0.3,
    'PnL': 0.25,
    'Sharpe Ratio': 0.2,
    'MDD': 0.15,  # We want to minimize MDD
    'Win Rate': 0.1
}

for metric in weights:
    if metric != 'MDD':
        metrics_df[metric] = (metrics_df[metric] - metrics_df[metric].min()) / (metrics_df[metric].max() - metrics_df[metric].min())
    else:
        metrics_df[metric] = (metrics_df[metric].max() - metrics_df[metric]) / (metrics_df[metric].max() - metrics_df[metric].min())

metrics_df['score'] = sum(metrics_df[metric] * weights[metric] for metric in weights)
metrics_df['rank'] = metrics_df['score'].rank(ascending=False)
top_20 = metrics_df.sort_values('score', ascending=False).head(20)

print(top_20)


                Port_IDs       ROI       PnL  Sharpe Ratio           MDD  \
97   4020204877254599680  1.000000  1.000000      0.374848  2.894981e-74   
76   3999240873283311617  0.651145  0.651145      0.488155  5.461103e-78   
37   3943533600390906881  0.174379  0.174379      0.629163  1.000000e+00   
63   3986814617275053313  0.340079  0.340079      0.629569  5.461103e-78   
99   4021669203289716224  0.459704  0.459704      0.513170  5.461103e-78   
17   3907081197088384000  0.359980  0.359980      0.476704  2.806752e-19   
51   3960874214179953664  0.378374  0.378374      0.476041  5.461422e-78   
108  4028701921959171840  0.355064  0.355064      0.480262  2.966901e-63   
16   3891020560590657281  0.180246  0.180246      0.839068  5.461103e-78   
137  4037121179310444801  0.232195  0.232195      0.785371  0.000000e+00   
149  4041860229502600193  0.173975  0.173975      1.000000  5.457683e-78   
100  4022565861939831809  0.314709  0.314709      0.504862  4.823183e-62   
49   3956076

In [ ]:
# 5. Deliverable
metrics_df.to_csv("account_metrics.csv", index=False)
top_20.to_csv("top_20_accounts.csv", index=False)
